In [27]:
from architector.io_core import geometries
import itertools
import pandas as pd
import numpy as np

geos = geometries()

In [28]:
df = pd.read_csv('ligtype_angle_reference.csv')

In [29]:
df

,ligtype,denticity,angle_1_mean,angle_1_95conf,angle_2_mean,angle_2_95conf,angle_3_mean,angle_3_95conf,angle_4_mean,angle_4_95conf,...,angle_32_mean,angle_32_95conf,angle_33_mean,angle_33_95conf,angle_34_mean,angle_34_95conf,angle_35_mean,angle_35_95conf,angle_36_mean,angle_36_95conf
0,bi_cis,2,83.450333,10.304849,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
1,tri_mer,3,161.464785,10.726856,82.597710,8.572949,79.508828,5.643762,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
2,hexa_octahedral,6,175.537700,5.439798,173.080704,5.769258,171.650579,6.295659,97.648141,4.937654,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
3,octa_trigonal_prismatic_triangle_face_bicapped,8,178.822923,1.477061,140.672652,4.741342,137.108868,4.792230,133.468580,5.508108,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
4,hexa_planar,6,174.830193,6.975056,171.413880,9.112972,168.867841,10.524867,122.492819,3.871297,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
5,bi_cis_bulky,2,82.753244,8.393160,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
6,penta_planar,5,148.448488,4.474695,147.261650,3.826581,142.907710,3.607654,140.856323,3.726230,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
7,bi_cis_chelating,2,50.674749,2.645427,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
8,tri_fac,3,89.269833,5.863737,86.504225,5.141199,83.899469,5.885095,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
9,penta_square_pyramidal,5,175.597817,4.306939,173.279477,4.367322,96.131640,2.237061,94.558539,2.081061,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000


In [30]:
mean_angle_labels = [x for x in df.columns.values if '_mean' in x] # Mean angle!
std_angle_labels = [x for x in df.columns.values if '_95conf' in x]

In [38]:
metal_coords = [0.0,0.0,0.0]

def get_angle(coord1,coord2,coord3):
    if isinstance(coord1,list):
        coord1=np.array(coord1)
    if isinstance(coord2, list):
        coord2 = np.array(coord2)
    if isinstance(coord3, list):
        coord3 = np.array(coord3)
    v1 = coord1-coord2
    v2 = coord3-coord2
    angle = np.degrees(np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))))
    return angle

def check_intercalation(coordat_list, all_coordats):
    coordats = np.array(coordat_list)
    if len(coordats) > 1:
        all_coordats = np.array(all_coordats)
        test_coordats = []
        for coord in all_coordats: # Remove matched coordatoms from full list.
            good = True
            for coordat in coordats:
                if np.linalg.norm(coord-coordat) < 1e-12:
                    good = False
            if good:
                test_coordats.append(coord)
        test_coordats = np.array(test_coordats)
        intercalating = False
        for pair in itertools.combinations(coordats,2): # Take pairs of coord atoms
            if get_angle(pair[0], metal_coords, pair[1]) < 145: # If the angle is less than 145 check for intercalation
                midvect = (pair[0] + pair[1]) # Take the midpoint vector between the two cord points
                for coordat in test_coordats: # Test the remaining coordintion sites
                    if get_angle(midvect,metal_coords,coordat) < 30: 
                        # If any of the points are within 30 degrees of the midpoint
                        # Flag as intercalating
                        intercalating = True
    else:
        intercalating = False
    return intercalating
    


outdict_rows = []
labels = []
angle_losses = []

for i,row in df.iterrows():
    outdict = dict()
    outdict_losses = dict()
    labels.append(row['ligtype'])
    cn_min = row['denticity']
    mean_angles = row[mean_angle_labels].values
#     max_angles = mean_angles + row[std_angle_labels].values*5
    max_angles = mean_angles + np.ones(36)*30 # Use looser criteron to get more points.
#     min_angles = mean_angles - row[std_angle_labels].values*5
    min_angles = mean_angles - np.ones(36)*30
    possible_geos = []
    for n in geos.cn_geo_dict.keys(): # get rid of geo types with less than min coordination
        if n >= cn_min:
            possible_geos += geos.cn_geo_dict[n]
    for core_type in possible_geos: # Define possible coordination environments and selected indices
        # Rule out pairs or sets where additional points fall between the selected indices.
        coordat_locs = geos.geometry_dict[core_type]
        cn = len(coordat_locs)
        all_coordat_inds = [x for x in itertools.combinations(range(len(coordat_locs)),cn_min)]
        all_coordat_combs = []
        for x in all_coordat_inds:
            tmp = []
            for k in x:
                tmp.append(coordat_locs[k])
            all_coordat_combs.append(tmp)
        for j,coordat_comb in enumerate(all_coordat_combs):
            angs = [
                get_angle(x[0],metal_coords,x[1]) for x in itertools.combinations(coordat_comb,2)
                ]
            angs = np.array(angs)[np.argsort(angs)[::-1]] # sort angles largest first!
            if cn_min < 9: # pad angles with zeros
                angs = np.pad(angs, (0,36-len(angs)), 'constant')
            if np.all(angs <= max_angles) & np.all(angs >= min_angles):
                loss = np.mean(np.abs(angs - mean_angles)) # MAE
                # Check for intercalating coordination points!!! -> don't use these!!!
                intercalating = check_intercalation(coordat_comb, coordat_locs)
                if (core_type in outdict) and (not intercalating):
                    temp = outdict[core_type].copy()
                    temp.append(all_coordat_inds[j])
                    outdict.update({core_type:temp})
                    tmp_losses = outdict_losses[core_type].copy()
                    tmp_losses.append(loss)
                    outdict_losses.update({core_type:tmp_losses}) # Save losses
                elif (not intercalating):
                    outdict.update({core_type:[all_coordat_inds[j]]})
                    outdict_losses.update({core_type:[loss]})
        ##### Sort outdict by losses
        for key in outdict.keys():
            tmp_inds = outdict[key].copy()
            tmp_losses = outdict_losses[key]
            order = np.argsort(tmp_losses) # Smallest loss first
            out = np.array(tmp_inds)[order]
            out = out.tolist()
            outdict[key] = out
    outdict_rows.append(outdict)
total_dict = dict(zip(labels,outdict_rows))

/var/folders/hm/58cd4wn126gg0_gbqn15c21800144l/T/ipykernel_18202/3201287515.py:12: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))))


In [40]:
total_dict.keys()

dict_keys(['bi_cis', 'tri_mer', 'hexa_octahedral', 'octa_trigonal_prismatic_triangle_face_bicapped', 'hexa_planar', 'bi_cis_bulky', 'penta_planar', 'bi_cis_chelating', 'tri_fac', 'penta_square_pyramidal', 'bi_cis_planar', 'hexa_trigonal_prismatic', 'tetra_planar', 'tri_mer_bent', 'penta_pyramidal', 'tetra_seesaw', 'octa_square_antiprismatic', 'tetra_trigonal_pyramidal', 'tetra_planar_bent', 'penta_planar_bent', 'hepta_capped_trigonal_prismatic', 'tetra_pyramidal', 'octa_cubic', 'hepta_5_2', 'hepta_pentagonal_bipyramidal', 'bi_trans', 'nona_capped_square_antiprismatic'])

In [51]:
total_dict['tri_fac']

{'trigonal_pyramidal': [[0, 1, 2]],
 'tetrahedral': [[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]],
 'seesaw': [[0, 1, 3], [0, 2, 3]],
 'square_pyramidal': [[0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4]],
 'trigonal_bipyramidal': [[0, 2, 3], [2, 3, 4]],
 'octahedral': [[0, 1, 3],
  [0, 1, 4],
  [0, 2, 3],
  [0, 2, 4],
  [1, 3, 5],
  [1, 4, 5],
  [2, 3, 5],
  [2, 4, 5]],
 'pentagonal_pyramidal': [[0, 1, 5],
  [0, 2, 3],
  [0, 3, 4],
  [0, 1, 2],
  [0, 4, 5]],
 'trigonal_prismatic': [[0, 1, 2], [3, 4, 5]],
 'pentagonal_bipyramidal': [[0, 1, 2],
  [0, 1, 5],
  [0, 2, 3],
  [0, 3, 4],
  [0, 4, 5],
  [1, 2, 6],
  [1, 5, 6],
  [2, 3, 6],
  [3, 4, 6],
  [4, 5, 6]],
 'capped_trigonal_prismatic': [[0, 1, 2],
  [1, 2, 6],
  [1, 4, 6],
  [2, 5, 6],
  [3, 4, 5],
  [4, 5, 6]],
 'capped_octahedral': [[0, 1, 3],
  [0, 1, 4],
  [0, 2, 3],
  [0, 2, 4],
  [1, 3, 5],
  [1, 4, 5],
  [2, 3, 5],
  [2, 4, 5]],
 'square_antiprismatic': [[0, 2, 3],
  [0, 1, 4],
  [1, 2, 3],
  [4, 5, 7],
  [5, 6, 7],
  [3, 5, 6],
  

In [53]:
total_dict['sandwich']

{'trigonal_pyramidal': [[0, 1, 2]],
 'tetrahedral': [[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]],
 'seesaw': [[0, 1, 3], [0, 2, 3]],
 'square_pyramidal': [[0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4]],
 'trigonal_bipyramidal': [[0, 2, 3], [2, 3, 4]],
 'octahedral': [[0, 1, 3],
  [0, 1, 4],
  [0, 2, 3],
  [0, 2, 4],
  [1, 3, 5],
  [1, 4, 5],
  [2, 3, 5],
  [2, 4, 5]],
 'pentagonal_pyramidal': [[0, 1, 5],
  [0, 2, 3],
  [0, 3, 4],
  [0, 1, 2],
  [0, 4, 5]],
 'trigonal_prismatic': [[0, 1, 2], [3, 4, 5]],
 'pentagonal_bipyramidal': [[0, 1, 2],
  [0, 1, 5],
  [0, 2, 3],
  [0, 3, 4],
  [0, 4, 5],
  [1, 2, 6],
  [1, 5, 6],
  [2, 3, 6],
  [3, 4, 6],
  [4, 5, 6]],
 'capped_trigonal_prismatic': [[0, 1, 2],
  [1, 2, 6],
  [1, 4, 6],
  [2, 5, 6],
  [3, 4, 5],
  [4, 5, 6]],
 'capped_octahedral': [[0, 1, 3],
  [0, 1, 4],
  [0, 2, 3],
  [0, 2, 4],
  [1, 3, 5],
  [1, 4, 5],
  [2, 3, 5],
  [2, 4, 5]],
 'square_antiprismatic': [[0, 2, 3],
  [0, 1, 4],
  [1, 2, 3],
  [4, 5, 7],
  [5, 6, 7],
  [3, 5, 6],
  

In [54]:
import pickle

In [55]:
with open('all_ref_geo_inds_dict.pkl','wb') as file1:
    pickle.dump(total_dict,file1)

In [56]:
with open('all_ref_geo_inds_dict.pkl','rb') as file2:
    alldict = pickle.load(file2)

In [25]:
a = np.array([0,1,2,3,4,5])

In [26]:
np.argsort(a)[::-1]

array([5, 4, 3, 2, 1, 0])

In [64]:
thd = geos.geometry_dict['tetrahedral']

In [65]:
np.linalg.norm(thd,axis=1)

array([2.        , 2.00011246, 2.00011246, 2.00009409])